## 4-2. BOW 기반의 카운트 벡터 생성

BOW의 실습을 위한 말뭉치로 NLTK가 제공하는 영화 리뷰를 사용한다. movie_reviews라는 이름으로 제공되는 이 말뭉치는, 2,000개의 영화 리뷰와 그 내용이 긍정적인지 부정적인지를 나타내는 감성 값이 함께 제공된다. movie_reviews는 fileids(), categories(), raw(), sents(), words() 등 주요 함수를 제공한다. 이를 통해 아래와 같이 영화 리뷰 문서의 id와 본문을 가져오고 문서의 내용과 토큰화 결과 등을 들여다본다.

In [1]:
import nltk
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\SystemsAnalysisLab\AppData\Roaming\nltk_data.
[nltk_data]     ..
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [2]:
from nltk.corpus import movie_reviews

print('# review count:', len(movie_reviews.fileids())) # 영화 리뷰 문서의 id를 반환
print()

print('# samples of file ids:', movie_reviews.fileids()[:10]) # id를 10개까지만 출력
print()

fileid = movie_reviews.fileids()[0] # 첫번째 문서의 id를 반환
print('# id of the first review:', fileid)
print()

print('# first review content:\n', movie_reviews.raw(fileid)[:200]) # 첫번째 문서의 내용을 200자까지만 출력
print()

print('# sentence tokenization result:', movie_reviews.sents(fileid)[:2]) # 첫번째 문서를 sentence tokenize한 결과 중 앞 두 문장
print()

print('# word tokenization result:', movie_reviews.words(fileid)[:20]) # 첫번째 문서를 word tokenize한 결과 중 앞 20개 단어

# review count: 2000

# samples of file ids: ['neg/cv000_29416.txt', 'neg/cv001_19502.txt', 'neg/cv002_17424.txt', 'neg/cv003_12683.txt', 'neg/cv004_12641.txt', 'neg/cv005_29357.txt', 'neg/cv006_17022.txt', 'neg/cv007_4992.txt', 'neg/cv008_29326.txt', 'neg/cv009_29417.txt']

# id of the first review: neg/cv000_29416.txt

# first review content:
 plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . 
w

# sentence tokenization result: [['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.'], ['they', 'get', 'into', 'an', 'accident', '.']]

# word tokenization result: ['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.', 'they', 'get', 'into', 'an']


위 결과를 요약해보면, 총 2,000개의 리뷰 문서가 있으며, fileid는 문자열로 되어있고, 그 안에 감성과 파일명이 포함돼 있는 것을 알 수 있다. movie_reviews.sents()의 결과를 보면, 먼저 문서를 문장 단위로 토큰화하고 그 후 각 문장을 단어 단위로 토큰화했으므로 결과가 리스트의 리스트로 되어 있음을 알 수 있다. 반면 movie_reviews.words()는 원문 전체에 대해 바로 단어 단위로 토큰화했으므로 결과가 하나의 리스트로 되어 있다. 

이제 BOW를 이용해 텍스트를 특성 벡터로 변환하는 방법을 정리해본다. 특성값은 단어의 빈도로 한다. 
  
1. 각 문서에 대해 앞서 배운 텍스트 전처리를 수행해 의미가 있는 최소 단위의 리스트로 변환한다. 앞서 배운 토큰화, 정규화, 품사 태깅 등의 방법을 모두 사용할 수도 있으나, 우선은 movie_reviews.words()가 제공하는 결과를 써 보기로 한다.
2. 특성 추출 대상이 되는 단어 집합인 특성 집합을 구성한다. 특성 집합은 텍스트 마이닝의 경우, 어휘 집합이라고 부르기도 한다. 말뭉치에 있는 모든 단어를 다 사용하는 것도 방법이 될 수 있으나, 자신이 정한 기준에 따라 단어들을 선별할 수도 있다. 예를 들어, 단어에 대해 빈도를 계산하고 빈도가 높은 상위 단어 n개만 사용할 수 있다.
3. 각 문서별로 특성 추출 대상 단어들에 대해 단어의 빈도를 계산해 특성 벡터를 추출한다. 단어의 빈도를 특성 값으로 사용해서 카운트 벡터라고도 부른다.

위 과정을 이제 기본적인 파이썬 함수들을 사용해 구현한다. 보통의 텍스트 마이닝 과정에서는 이 부분을 직접 구현하지 않고 사이킷런과 같은 라이브러리를 쓰지만, 여기에서는 BOW에 대해 명확히 이해하기 위해 직접 구현해본다.

먼저 각 문서에 대한 토큰화 결과들로 리스트를 만들기 위해 아래 코드를 실행한다. fileids()를 이용해 모든 문서의 id를 가져오고 각 id들에 대해 words()로 토큰화 결과를 가져와 리스트를 만든다.

In [3]:
documents = [list(movie_reviews.words(fileid)) for fileid in movie_reviews.fileids()]

print(documents[0][:50]) # 첫째 문서의 앞 50개 단어 출력

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.', 'they', 'get', 'into', 'an', 'accident', '.', 'one', 'of', 'the', 'guys', 'dies', ',', 'but', 'his', 'girlfriend', 'continues', 'to', 'see', 'him', 'in', 'her', 'life', ',', 'and', 'has', 'nightmares', '.', 'what', "'", 's', 'the', 'deal', '?', 'watch']


이제 특성 집합을 만들기 위해 딕셔너리를 사용해 단어별로 말뭉치 전체에서의 빈도를 계산하고, 빈도가 높은 단어부터 정렬한 후에 빈도수 상위 20개의 단어를 출력한다.

In [4]:
word_count = {}
for text in documents:
    for word in text:
        word_count[word] = word_count.get(word, 0) + 1

sorted_features = sorted(word_count, key=word_count.get, reverse=True)
for word in sorted_features[:10]:
    print(f"count of '{word}': {word_count[word]}", end=", ")

count of ',': 77717, count of 'the': 76529, count of '.': 65876, count of 'a': 38106, count of 'and': 35576, count of 'of': 34123, count of 'to': 31937, count of ''': 30585, count of 'is': 25195, count of 'in': 21822, 

위 결과를 보면, ',', 'the', 'a'와 같이 의미적으로 쓸모없는 단어만 빈도가 높은 것 같아 보인다. 이를 해결하기 위해 정규표현식으로 다시 토큰화하고 앞의 과정을 반복해본다. 이를 위해 먼저 raw()를 이용해 원문을 가져와 documents를 만들고, 이에 대해 토큰화를 한다. 하는 김에 NLTK 불용어 사전을 이용해 불용어도 제거한다.

In [5]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

tokenizer = RegexpTokenizer("[\w']{3,}") # 정규표현식으로 토크나이저를 정의
english_stops = set(stopwords.words('english')) # 영어 불용어를 가져옴

documents = [movie_reviews.raw(fileid) for fileid in movie_reviews.fileids()] # words() 대신 raw()로 원문을 가져옴

tokens = [[token for token in tokenizer.tokenize(doc) if token not in english_stops] for doc in documents] # stopwords의 적용과 토큰화를 동시에 수행

word_count = {}
for text in tokens:
    for word in text:
        word_count[word] = word_count.get(word, 0) + 1

sorted_features = sorted(word_count, key=word_count.get, reverse=True)

print('num of features:', len(sorted_features))
for word in sorted_features[:10]:
    print(f"count of '{word}': {word_count[word]}", end=", ")

num of features: 43030
count of 'film': 8935, count of 'one': 5791, count of 'movie': 5538, count of 'like': 3690, count of 'even': 2564, count of 'time': 2409, count of 'good': 2407, count of 'story': 2136, count of 'would': 2084, count of 'much': 2049, 

이렇게 추출된 단어 혹은 특성은 43,030개이고 상위 빈도를 봤을 때 이전보다는 훨씬 나아 보인다. 이 특성들을 다 사용해도 좋지만, 상위 빈도수를 가지는 단어 천 개만 추출해서 최종적으로 문서를 표현할 특성으로 사용하기로 한다. 이렇게 하면 단어로 구성된 특성 집합이 완성된다. 이때 중요한 것은, 특성 집합에는 단어의 순서가 있으며 이 순서에 따라 문서의 카운트 벡터 값이 결정된다는 것이다.

In [6]:
word_features = sorted_features[:1000] # 빈도가 높은 상위 1000개의 단어만 추출해 features를 구성

이제 주어진 문서를 특성 벡터, 즉 카운트 벡터로 변환하는 함수를 만든다. 함수가 제대로 작동하는지 알아보기 위해 아래와 같이 입력과 출력의 예를 만들어서 테스트해본다.

- 특성 집합 예제(word_features_ex): ['one', 'two', 'teen', 'couples', 'solo']
- 주어진 문서의 토큰화 결과: ['two', 'two', 'couples']
- 변환된 카운트 벡터 결과: [0, 2, 0, 1, 0]

주어진 문서에서 'two'가 2회, 'couples'가 1회 사용됐으므로, 전체 특성 리스트와 맞춰봤을 때 벡터에서 둘째와 넷째 값이 각각 2와 1이 되어야 한다. 이때 특성 집합 word_features에 있는 단어들은 빈도에 따라 순서가 정해져 있으므로 카운트 벡터에서는 word_features에 있는 단어의 순서에 따라 빈도만 기록하면 된다.

In [7]:
# 주어진 document를 feature로 변환하는 함수, word_features를 사용
def document_features(document, word_features):
    word_count = {}
    for word in document: # document에 있는 단어들에 대해 빈도수를 먼저 계산
        word_count[word] = word_count.get(word, 0) + 1
    
    features = []
    for word in word_features: # word_features의 단어에 대해 계산된 빈도수를 feature에 추가
        features.append(word_count.get(word, 0)) # 빈도가 없는 단어는 0을 입력
    
    return features

word_features_ex = ['one', 'two', 'teen', 'couples', 'solo']
doc_ex = ['two', 'two', 'couples']
print(document_features(doc_ex, word_features_ex))

[0, 2, 0, 1, 0]


테스트가 잘 작동하는 것을 확인했으므로 이제 전체 리뷰 집합에 대해 적용하고, 추출된 특성의 수와 첫째 리뷰 문서에 대한 특성 집합 중 앞부분 20개만 출력해본다. 이때 벡터에 있는 값에 대해 어떤 단어의 빈도인지 알기 위해 순서를 맞춰 word_features의 단어와 함께 출력한다.

In [8]:
feature_sets = [document_features(d, word_features) for d in tokens]

# 첫째 feature set의 내용을 앞 20개만 word_features의 단어와 함께 출력
for i in range(20):
    print(f'({word_features[i]}, {feature_sets[0][i]})', end=', ')

(film, 5), (one, 3), (movie, 6), (like, 3), (even, 3), (time, 0), (good, 2), (story, 0), (would, 1), (much, 0), (also, 1), (get, 3), (character, 1), (two, 2), (well, 1), (first, 0), (characters, 1), (see, 2), (way, 3), (make, 5), 

위 결과에서 word_features의 단어는 빈도가 높은 순서대로 정렬되어 있으므로, 앞 20개 단어에는 카운트 값이 0인 단어가 많이 없는 것을 볼 수 있다. 그러나 뒤로 갈수록 값이 0인 단어들이 많아질 것으로 예측할 수 있고, 이를 확인하기 위해 아래와 같이 마지막 20개 값을 출력해보면 모두 0인 것을 볼 수 있다.

In [9]:
print(feature_sets[0][-20:]) # feature set의 뒤 20개만 출력 

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


이상과 같은 방법으로 말뭉치에 있는 모든 문서들을 카운트 벡터로 변환할 수 있다. 카운트 벡터의 구성과 특징을 요약하자면 카운트 벡터는 말뭉치 전체의 단어 집합 혹은 그중에서 선별한 단어 집합에 대한 단어들의 빈도로 이루어져 있으며, 매우 희소하다는 특징이 있다.